In [207]:
# You only need to change values in this cell and then run the whole script 

image_path = 'T4-24 (SubtypeB)/T4-24.tif'
Roi_path = 'T4-24 (SubtypeB)/RoiSet.zip'
save_path = 'T4-24 (SubtypeB)/'
save_name = 'T4_24'
normalize = True
sig = 1 # this number define the gaussian blur radius. If 1, then no blur is applied

# entry_point = (38,185,225) #entry point of the neuron (Z,Y,X)

if save_path[-1] != '/':
    save_path += '/'

In [208]:
#imports
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
from tqdm import tqdm
import pandas as pd
from read_roi import read_roi_zip as co_zip
import cv2
from skimage.filters import gaussian, threshold_otsu
from scipy import ndimage

In [209]:
def mask_image(volume, return_mask = False ,sig = 1):
    image = volume.copy()
    image = image.astype('float32')
    image -= image.min()
    image /= image.max()
    blur = gaussian(image, sigma=sig)
    t = threshold_otsu(blur)
    mask = blur > t
    mask = np.array(mask, dtype=bool)
    if return_mask == False:
        image[mask==False] = 0
        mask = image
    return mask

def segment_3D(image):
    s = ndimage.generate_binary_structure(len(image.shape), len(image.shape))
    labeled_array, num_labels = ndimage.label(image, structure=s)
    labels = np.unique(labeled_array)
    labels = labels[labels!=0]
    neu_sizes = {}
    for l in labels:
        neu_sizes[l] = (labeled_array == l).sum()/(labeled_array == l).max()
    # sorted_sizes = sorted(neu_sizes.items(), key=operator.itemgetter(1), reverse=True)
    max_label = max(neu_sizes, key=neu_sizes.get)
    neuron = labeled_array.copy()
    neuron [neuron!=max_label] = 0       
    return neuron

def col_occupancy(neuron, cols_zip, normalize=True, plot=True, save=True, save_path='', save_file=''):
    cols_occ = {}
    for key, val in tqdm(cols_zip.items()):
        if val['type'] == 'oval':
            x0 = val['left']+int(val['width']/2); a = int(val['width']/2)  # x center, half width                                       
            y0 = val['top']+int(val['height']/2); b = int(val['height']/2)  # y center, half height                                      
            x = np.linspace(0, neuron.shape[-1],neuron.shape[-1])  # x values of interest
            y = np.linspace(0, neuron.shape[-2],neuron.shape[-2])[:,None]  # y values of interest, as a "column" array
            column = ((x-x0)/a)**2 + ((y-y0)/b)**2 <= 1  # True for points inside the ellipse
            column = column.astype(int)
        elif val['type'] == 'freehand':
            rois = np.array(list(zip(val['x'],val['y'])))
            rois = rois.astype(int)
            column = np.zeros([neuron.shape[-2],neuron.shape[-1]])
            column = cv2.fillPoly(column, pts =[rois], color=(255,255,255))
            column[column>0] = 1
        col_filter = np.broadcast_to(column, neuron.shape)
        col_size = col_filter.sum()
        nue_sub = col_filter * neuron # pixels occupied by neuron in the column
        cols_occ[key] = nue_sub.sum()/col_size
    cols_occ = pd.Series(cols_occ)
    if normalize:
        cols_occ = cols_occ/cols_occ.max()
    if save == True:
        if save_file == '':
            save_file = "col_occupancy.csv"
        csv_file = save_path+save_file
        if '.csv' not in csv_file:
            csv_file +='.csv'
        cols_occ.to_csv(csv_file, sep=';')

    return cols_occ

In [210]:
image = tif.imread(image_path)

In [211]:
mask = mask_image(image, return_mask = False ,sig = sig)
mask = segment_3D(mask)
neuron = image.copy()
neuron[mask==0] = 0
tif.imwrite(save_path+save_name+'_mask.tif', neuron, 
            imagej=True, resolution=(1./0.076, 1./0.076),
            metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10,'axes': 'ZYX'})

In [212]:
cols_zip = rois = co_zip(Roi_path)
neuron[neuron!=0] = 1

In [213]:
T4_4b_Cols_Ai = col_occupancy(neuron, 
                              cols_zip, 
                              normalize=normalize, 
                              save=True, 
                              save_path=save_path, 
                              save_file=save_name)

100%|██████████| 15/15 [00:01<00:00,  7.78it/s]
